In [7]:
APItoken = '2828a9be6ef0add8b6c59bb2e36121002bb3898b247ca95e9d17796f36bd78d5742c07620c30f2884dff23532118a155cc702ee3a6b8b215eba4a8fc88fe3e31'

IBMQ.save_account(APItoken, overwrite=True)

In [8]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.circuit import Gate
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.circuit.library import QFT, XGate
from qiskit.extensions import UnitaryGate
from scipy.linalg import expm
#Loading your IBM Quantum account(s)
provider = IBMQ.load_account()


In [9]:
def sim_AD(num_qubits: int, damped_qubit: int, theta: float):
    
    #Defines a sim-AD block
    #
    #Inputs: 
    #num_qubits: size of the quantum registers
    #damped_qubit: position of the qubit that will undergo damping
    #theta: parameter of the CRy gate
    #
    #Output:
    #Circuit with phase and environment quantum registers
    
    qr_phase = QuantumRegister(num_qubits, 'phase')
    qr_env = QuantumRegister(num_qubits, 'env')
    
    qc = QuantumCircuit(qr_env, qr_phase)
    
    qc.cry(theta, control_qubit = qr_phase[damped_qubit], target_qubit= qr_env[damped_qubit])
    
    qc.cnot(qr_env[damped_qubit], qr_phase[damped_qubit])
    
    
    return qc.to_gate(label = 'sim-AD_{}[{}]'.format(damped_qubit, theta))

def sim_AD_chain(num_qubits: int, thetas: list):
    
    #Defines a sim-AD chain (no measurement included)
    #
    #Inputs: 
    #num_qubits: size of the quantum registers
    #theta: parameters of the CRy gates, ordered according to the qubits in the registers
    #
    #Output:
    #Circuit with phase and environment quantum registers
    
    qr_phase = QuantumRegister(num_qubits, 'phase')
    qr_env = QuantumRegister(num_qubits, 'env')
    qc = QuantumCircuit(qr_env, qr_phase)
    
    for j in range(num_qubits):
        sim_AD_gate_j = sim_AD(num_qubits = num_qubits, damped_qubit = j, theta = thetas[j])
        qc.append(sim_AD_gate_j, qr_env[:] + qr_phase[:])
    
    return qc

def QPE_circuit(work_qubits: int, phase_qubits: int, MGate: Gate):
    
    #Defines the QPE circuit
    #
    #Inputs: 
    #work_qubits: size of the work register
    #phase_qubits: size of the phase register
    #MGate: Gate object, operator M as in the article - Section (3.2.2)
    #theta: parameters of the CRy gates, ordered according to the qubits in the registers
    #
    #Output:
    #Circuit with work amd phase quantum registers

    
    qr_work = QuantumRegister(work_qubits, 'work')
    qr_phase = QuantumRegister(phase_qubits, 'phase')
    
    qc = QuantumCircuit(qr_phase, qr_work)
    
    qc.h(qr_phase[:])
    
    for j in range(phase_qubits):
        
        MGate_j = MGate.power(2**j)
        MGate_j.label = 'M^{}'.format(2**j)
        
        qc.append(MGate_j.control(), [qr_phase[j]] + qr_work[:])
        
    qft = QFT(num_qubits=phase_qubits, inverse=True, insert_barriers=False)
    
    qc.append(qft, qr_phase[:])
    
    return qc

def Mgate_generate(A: np.array, normA: float, strictly_positive = True, L = 0):
    
    #Defines the M Gate as in article - Section (3.2.2) or Appendix B (Eq. 11)
    #
    #Inputs: 
    #A: evolution matrix from original HDLE
    #normA: 2-norm of A
    #strictly_positive: boolean for switching between both formulations
    #L: Hilbert space dimension for phase register (only required if strictly_positive = False)
    #
    #Output:
    #Gate object
    
    
    if strictly_positive:
        Ap = A/normA
    else:
        Ap = ((L-1)/(2*L))*A/normA + ((L+1)/(2*L))*np.identity(A.shape[0]) 
        
    M = expm(-2*(0+1j)*np.pi*Ap)
        
    MGate = UnitaryGate(M, label = 'M')
    
    return MGate

def thetas_generate(t: float, L:int, normA: float, strictly_positive = True): #According to eq. 6
    
    #Defines the theta parameters as in:
    #dissertation - Eq. 3.6 or Eq. 3.17 
    #article: Eq. 6 or 13
    #
    #Inputs: 
    #t: time variable from original HDLE
    #L: Hilbert space dimension for phase register
    #normA: 2-norm of A
    #strictly_positive: boolean for switching between both formulations
    #
    #Output:
    #Numpy array with parameter values

    
    thetas = []
    
    if strictly_positive:
        for k in range(int(np.log2(L))):
            arg_k = 2*np.exp(-normA*2**(k+1)*t/L)-1 # Eq. 3.6 or 6
            theta_k = np.arccos(arg_k)
            thetas.append(theta_k)
    else:
        for k in range(int(np.log2(L))):
            arg_k = 2*np.exp(-normA*2**(k+2)*t/(L-1))-1 # Eq. 3.17 or 13
            theta_k = np.arccos(arg_k)
            thetas.append(theta_k)
        
        
    return np.array(thetas)

def sim_AD_HDLE(A:np.array, t:int, x0: np.array, L: int, strictly_positive = False):
    
    #Defines complete circuit (no measurements) for solving HDLE as in
    #dissertation: Figure 4
    #article: Figure 1
    #
    #Inputs: 
    #A: evolution matrix from original HDLE
    #t: time variable from original HDLE
    #x0: initial condition vector from original HDLE
    #L: Hilbert space dimension for phase register
    #strictly_positive: boolean for switching between both formulations
    #
    #Output:
    #Numpy array with parameter values
    
    
    work_qubits = int(np.log2(A.shape[0]))
    phase_qubits = int(np.log2(L))
    
    qr_env = QuantumRegister(phase_qubits, 'env')
    qr_phase = QuantumRegister(phase_qubits, 'phase')
    qr_work = QuantumRegister(work_qubits, 'work')
    
    qc = QuantumCircuit(qr_work, qr_phase, qr_env)
    
    x0ket = x0/np.linalg.norm(x0,2)
    qc.initialize(x0ket, qr_work[:])
    
    qc.barrier()
    
    normA = np.linalg.norm(A, 2)
    MGate = Mgate_generate(A, normA, strictly_positive, L)
    
    qc = qc.compose(QPE_circuit(work_qubits, phase_qubits, MGate), qr_phase[:]+ qr_work[:])
    qc.barrier()
    
    thetas = thetas_generate(t, L, normA, strictly_positive)
    
    qc += sim_AD_chain(phase_qubits, thetas).decompose()
    
    qc.barrier()
    
    qc = qc.compose(QPE_circuit(work_qubits, phase_qubits, MGate).inverse(), qr_phase[:]+ qr_work[:])
    
    
    return qc

In [11]:

A = np.array(((0, -2), (-2, 0)))
x0 = np.array((3, 1)) 
t = 0.05
L = 2

work_qubits = 1
phase_qubits = int(np.log2(L))

qc_list = []
for i in [1, 10]:
    
    qr_env = QuantumRegister(phase_qubits, 'env')
    qr_phase = QuantumRegister(phase_qubits, 'phase')
    qr_work = QuantumRegister(work_qubits, 'work')
    qr_aux = QuantumRegister(work_qubits, 'aux')
        
    cr_env = ClassicalRegister(phase_qubits, 'c_env')
    cr_aux = ClassicalRegister(work_qubits, 'c_aux')
        
    qc = QuantumCircuit(qr_work, qr_phase, qr_env, qr_aux, cr_env, cr_aux)
        
    qc = qc.compose(sim_AD_HDLE(A, i*t, x0, L, strictly_positive = False), qr_work[:] + qr_phase[:] + qr_env[:])
        
    observable = XGate()#UnitaryGate(np.array(((0, 1), (1, 0))), label = 'X')
    controlled_obs = observable.control(work_qubits)
        
    qc.h(qr_aux[:])    
    qc = qc.compose(controlled_obs, qr_aux[:] + qr_work[:])  
    qc.h(qr_aux[:])
        
    qc.measure(qr_env[:] + qr_aux[:], cr_env[:] + cr_aux[:])

    qc_list.append(qc)

provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_quito')

qc05_naive = transpile(qc_list[0], backend)
qc5_naive =  transpile(qc_list[1], backend)

layout = [3, 1, 2, 4]
qc05_opt = transpile(qc_list[0], backend, initial_layout = layout)
qc5_opt =  transpile(qc_list[1], backend, initial_layout = layout)
    
qc_transpiled_list = [qc05_naive, qc05_opt, qc5_naive, qc5_opt]
    
job = backend.run(qc_transpiled_list, shots = 8192)
    
result = job.result()
counts = job.result().get_counts()

import json

with open('get_counts.txt', 'w') as f:
    for element in counts:
        f.write(json.dumps(element))
print('done')

done


In [19]:
naive05 = {"0 0": 4178, "0 1": 1786, "1 0": 2008, "1 1": 220}
opt05 = {"0 0": 4324, "0 1": 1996, "1 0": 1681, "1 1": 191}
naive5 = {"0 0": 955, "0 1": 4797, "1 0": 1469, "1 1": 971}
opt5 = {"0 0": 724, "0 1": 5527, "1 0": 1476, "1 1": 465}

all_counts = [naive05, opt05, naive5, opt5]

In [20]:
success_str = ' 0'

test_result = []
for counts in all_counts:
     success_meas = []
     for meas in counts.keys():
         if success_str in meas:
             success_meas.append(meas)
           
    
     success_count = { meas: counts[meas] for meas in success_meas }
   
     expX = (1*success_count['0'+success_str] - 1*success_count['1'+success_str])/(success_count['0'+success_str] + success_count['1'+success_str])
   
     inv_sign_flag = False
     if expX<0:
         inv_sign_flag = True
              
     test_result.append({'success_count': success_count, 'expX': expX, 'inv_sign_flag': inv_sign_flag})
              

In [21]:
test_result

[{'success_count': {'0 0': 4178, '1 0': 2008},
  'expX': 0.3507921112188813,
  'inv_sign_flag': False},
 {'success_count': {'0 0': 4324, '1 0': 1681},
  'expX': 0.4401332223147377,
  'inv_sign_flag': False},
 {'success_count': {'0 0': 955, '1 0': 1469},
  'expX': -0.21204620462046206,
  'inv_sign_flag': True},
 {'success_count': {'0 0': 724, '1 0': 1476},
  'expX': -0.3418181818181818,
  'inv_sign_flag': True}]